In [1]:

# Importamos NumPy con alias 'np' para operaciones numéricas y manipulación de matrices
import numpy as np
# Importamos pandas con alias 'pd' para manejo eficiente de datos tabulares
import pandas as pd
# Importamos 'datetime' y 'time' para trabajar con fechas y horas
from datetime import datetime, time
# Cargamos la extensión autoreload en Jupyter Notebook, útil para desarrollo interactivo
%load_ext autoreload
# Configuramos autoreload para recargar automáticamente todos los módulos antes de ejecutar el código
%autoreload 2
# Importamos el módulo warnings para gestionar las advertencias
import warnings
# Configuramos el filtro de advertencias para ignorarlas y mantener el código limpio de mensajes no deseados
warnings.filterwarnings("ignore")

Funcion verificar_tipos_datos

La función verificar_tipo_datos toma un DataFrame de Pandas como entrada y realiza un análisis detallado de cada columna. Genera un resumen que incluye el nombre de la columna, el tipo de datos, el porcentaje de valores no nulos, el porcentaje de valores nulos y la cantidad de valores nulos para cada columna. Además, la función imprime el total de valores nulos en todo el DataFrame y muestra las columnas con valores nulos ordenadas de mayor a menor según el porcentaje de valores nulos. El resultado es un DataFrame que proporciona una visión detallada de la calidad de los datos en el DataFrame original.

In [2]:
def verificar_tipo_datos(df):
    # Diccionario para almacenar información sobre cada columna
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    # Iterar sobre todas las columnas del DataFrame df
    for columna in df.columns:
        # Calcular el porcentaje de valores no nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100

        # Agregar información al diccionario
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    # Crear un DataFrame con la información recopilada
    df_info = pd.DataFrame(mi_dict)

    # Calcular el total de valores nulos en el DataFrame
    total_nulos = df.isnull().sum().sum()

    # Imprimir el total de valores nulos
    print(f'Total de valores nulos en el DataFrame: {total_nulos}')

    # Mostrar las columnas con valores nulos y sus respectivos porcentajes (ordenadas de mayor a menor)
    columnas_con_nulos = df_info[df_info["nulos"] > 0][["nombre_campo", "nulos_%"]].sort_values(by="nulos_%", ascending=False)
    if not columnas_con_nulos.empty:
        print("\nColumnas con valores nulos y sus porcentajes:")
        print(columnas_con_nulos)

    # Devolver el DataFrame con la información de cada columna
    return df_info

Verificar duplicados por columnas

La siguiente función proporciona una herramienta util para identificar y ordenar las filas duplicadas de un DataFrame de Pandas en función de los valores de una columna específica. En nuestro caso puede ser útil para el análisis de datos cuando se desea examinar y manejar duplicados en función de una columna particular.

In [3]:

def verifica_duplicados_por_columna(df, columna):

    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"

    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted

Obtener tipo de datos

Esta función, llamada obtener_tipos_de_datos, toma un DataFrame de pandas como entrada y devuelve un nuevo DataFrame que contiene información sobre los tipos de datos presentes en cada columna del DataFrame original.

In [4]:
def obtener_tipos_de_datos(dataframe):
    # Inicializa una lista vacía para almacenar la información de tipos de datos
    tipos_de_datos = []

    # Itera sobre las columnas del DataFrame
    for columna in dataframe.columns:
        # Para cada columna, crea una lista de tipos de datos de los elementos en esa columna
        tipos = [str(type(dato)) for dato in dataframe[columna]]

        # Convierte la lista de tipos a un conjunto para eliminar duplicados y luego vuelve a convertir a lista
        tipos_unicos = list(set(tipos))

        # Agrega un diccionario a la lista tipos_de_datos con el nombre de la columna y los tipos de datos únicos
        tipos_de_datos.append({
            'Columna': columna,
            'Tipo de dato': tipos_unicos
        })

    # Crea un nuevo DataFrame con la información de tipos_de_datos
    resultado = pd.DataFrame(tipos_de_datos)

    # Devuelve el DataFrame resultante
    return resultado

Converir tiempo

Esta función es flexible y trata de convertir la entrada en un objeto de tiempo, ya sea a partir de una cadena con formato específico o extrayendo el componente de tiempo de un objeto de fecha y hora. Si no puede realizar la conversión, devuelve None o el valor original, dependiendo del caso.

In [5]:
# Definimos una función para convertir diferentes tipos de entradas a un objeto de tiempo
def convertir_tiempo(x):
    # Verificamos si la entrada es una cadena de texto
    if isinstance(x, str):
        try:
            # Intentamos convertir la cadena de texto a un objeto de tiempo y lo retornamos
            return datetime.strptime(x, "%H:%M:%S").time()
        except ValueError:
            # Si hay un error en la conversión, retornamos None
            return None
    # Verificamos si la entrada es un objeto datetime
    elif isinstance(x, datetime):
        # Convertimos el objeto datetime a su correspondiente objeto de tiempo y lo retornamos
        return x.time()
    # Si la entrada no es ni cadena de texto ni datetime, retornamos la entrada sin modificar
    return x

Imputa valor frecuente

Esta función reemplaza los valores "SD" con NaN en la columna especificada, luego calcula el valor más frecuente en esa columna y utiliza ese valor para imputar los valores faltantes (NaN).

In [6]:
def imputa_valor_frecuente(df, columna):

    # Se reemplaza "SD" con NaN en la columna
    df[columna] = df[columna].replace('SD', pd.NA)

    # Se calcula el valor más frecuente en la columna
    valor_mas_frecuente = df[columna].mode().iloc[0]
    print(f'El valor mas frecuente es: {valor_mas_frecuente}')

    # Se imputan los valores NaN con el valor más frecuente
    df[columna].fillna(valor_mas_frecuente, inplace=True)

Imputa edad media segun sexo

Esta función reemplaza los valores "SD" con NaN en la columna 'Edad', calcula la edad promedio para cada grupo de género (Femenino y Masculino), imprime los promedios calculados y luego llena los valores faltantes en la columna 'Edad' utilizando el promedio correspondiente al género al que pertenece cada fila en el DataFrame.

In [7]:
def imputa_edad_media_segun_sexo(df):

    # Se reemplaza "SD" con NaN en la columna 'edad'
    df['Edad'] = df['Edad'].replace('SD', pd.NA)

    # Se calcula el promedio de edad para cada grupo de género
    promedio_por_genero = df.groupby('Sexo')['Edad'].mean()
    print(f'La edad promedio de Femenino es {round(promedio_por_genero["FEMENINO"])} y de Masculino es {round(promedio_por_genero["MASCULINO"])}')

    # Se llenan los valores NaN en la columna 'edad' utilizando el promedio correspondiente al género
    df['Edad'] = df.apply(lambda row: promedio_por_genero[row['Sexo']] if pd.isna(row['Edad']) else row['Edad'], axis=1)
    # Lo convierte a entero
    df['Edad'] = df['Edad'].astype(int)

In [8]:
# Cargamos el conjunto de datos de homicidios desde un archivo Excel específico, indicando la hoja 'HECHOS'
df_hechos = pd.read_excel(r'C:\Users\Moises\Desktop\Proyecto individual 2\archivos\homicidios.xlsx', sheet_name='HECHOS')
# Seleccionamos una muestra aleatoria de 5 registros del DataFrame para revisión preliminar, asegurando la reproducibilidad con un estado aleatorio fijo
df_hechos.sample(5,random_state=5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
197,2017-0069,1,2017-05-24,2017,5,24,14:15:00,14,AV. GRAL. PAZ Y ULRICO SCHMIDL,GRAL PAZ,...,NaN,"SCHMIDL, ULRICO","PAZ, GRAL. AV. y SCHMIDL, ULRICO",9,Point (94644.00300729 96592.22735507),-58.52173071,-34.65997463,MOTO-SD,MOTO,SD
672,2021-0074,1,2021-09-20,2021,9,20,15:00:00,15,"PAZ, GRAL. AV. Y BALBIN, RICARDO, DR. AV.",GRAL PAZ,...,NaN,"BALBIN, RICARDO, DR. AV.","PAZ, GRAL. AV. y BALBIN, RICARDO, DR. AV.",12,Point (96563.66494817 108815.73881056),-58.50073810,-34.54979510,MULTIPLE,MOTO,MULTIPLE
232,2017-0110,1,2017-09-06,2017,9,6,14:15:00,14,LAVALLE Y AV. LEANDRO N ALEM,AVENIDA,...,NaN,"ALEM, LEANDRO N. AV.","LAVALLE y ALEM, LEANDRO N. AV.",1,Point (108513.43549456 103032.95078343),-58.37048800,-34.60189492,PEATON-PASAJEROS,PEATON,PASAJEROS
673,2021-0075,1,2021-09-21,2021,9,21,14:30:00,14,SALTA Y BRASIL,CALLE,...,NaN,BRASIL,SALTA y BRASIL,1,Point (107380.60401316 100153.47427104),-58.38281301,-34.62786038,PEATON-CARGAS,PEATON,CARGAS
369,2018-0095,1,2018-08-26,2018,8,26,00:59:00,0,"Lacarra Av. y Dellepiane, Luis, Tte. Gral.",AVENIDA,...,NaN,"DELLEPIANE, LUIS, TTE. GRAL.","LACARRA AV. y DELLEPIANE, LUIS, TTE. GRAL.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,AUTO-AUTO,AUTO,AUTO


In [9]:
# Cargamos el conjunto de datos de víctimas desde un archivo Excel, especificando la hoja 'VICTIMAS'
df_victimas = pd.read_excel(r'C:\Users\Moises\Desktop\Proyecto individual 2\archivos\homicidios.xlsx', sheet_name='VICTIMAS')
# Seleccionamos una muestra aleatoria de 5 registros del DataFrame para realizar una revisión preliminar, usando un estado aleatorio fijo para reproducibilidad
df_victimas.sample(5,random_state=5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
521,2019-0086,2019-10-15,2019,10,15,CONDUCTOR,MOTO,MASCULINO,29,2019-10-15 00:00:00
698,2021-0079,2021-10-25,2021,10,25,PEATON,PEATON,MASCULINO,SD,2021-10-25 00:00:00
300,2018-0015,2018-02-06,2018,2,6,CONDUCTOR,MOTO,MASCULINO,28,2018-02-06 00:00:00
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
23,2016-0034,2016-03-12,2016,3,12,PEATON,PEATON,MASCULINO,52,2016-03-12 00:00:00


Analizamos la pestaña Hechos con el dataframe df_hechos

Información del dataframe

Vemos que tenemos 696 registros con 21 columnas

In [10]:
# Obtenemos las dimensiones del DataFrame df_hechos, lo cual incluye el número de filas (registros) y columnas (atributos)
df_hechos.shape

(696, 21)

In [11]:

# Se observan las columnas de dataset
df_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

Vemos que algunas columnas estan en mayúsculas y minisculas, se decide estandarizar los titulos en las columnas.

In [12]:

# Se coloca la primera en mayúscula
df_hechos.columns = [x.capitalize() for x in df_hechos.columns]
# Se reemplazan los guiones por espacios
df_hechos.columns = df_hechos.columns.str.replace('_', ' ')
# Se renombran algunas columnas
df_hechos = df_hechos.rename(columns={'N° victimas': 'Cantidad víctimas',
                                                      'Aaaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Hh':'Hora entera',
                                                      'Xy (caba)':'XY (CABA)',
                                                      'Victima': 'Víctima'})

Vemos como queda nustro dataframe con las columnas estandarizadas

In [13]:
# Ejemplo del dataframe df_hechos con 5 filas y el número de semilla n° 5 para ver siempre las mismas filas
df_hechos.sample(5,random_state=5)

,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,...,Altura,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
197,2017-0069,1,2017-05-24,2017,5,24,14:15:00,14,AV. GRAL. PAZ Y ULRICO SCHMIDL,GRAL PAZ,...,NaN,"SCHMIDL, ULRICO","PAZ, GRAL. AV. y SCHMIDL, ULRICO",9,Point (94644.00300729 96592.22735507),-58.52173071,-34.65997463,MOTO-SD,MOTO,SD
672,2021-0074,1,2021-09-20,2021,9,20,15:00:00,15,"PAZ, GRAL. AV. Y BALBIN, RICARDO, DR. AV.",GRAL PAZ,...,NaN,"BALBIN, RICARDO, DR. AV.","PAZ, GRAL. AV. y BALBIN, RICARDO, DR. AV.",12,Point (96563.66494817 108815.73881056),-58.50073810,-34.54979510,MULTIPLE,MOTO,MULTIPLE
232,2017-0110,1,2017-09-06,2017,9,6,14:15:00,14,LAVALLE Y AV. LEANDRO N ALEM,AVENIDA,...,NaN,"ALEM, LEANDRO N. AV.","LAVALLE y ALEM, LEANDRO N. AV.",1,Point (108513.43549456 103032.95078343),-58.37048800,-34.60189492,PEATON-PASAJEROS,PEATON,PASAJEROS
673,2021-0075,1,2021-09-21,2021,9,21,14:30:00,14,SALTA Y BRASIL,CALLE,...,NaN,BRASIL,SALTA y BRASIL,1,Point (107380.60401316 100153.47427104),-58.38281301,-34.62786038,PEATON-CARGAS,PEATON,CARGAS
369,2018-0095,1,2018-08-26,2018,8,26,00:59:00,0,"Lacarra Av. y Dellepiane, Luis, Tte. Gral.",AVENIDA,...,NaN,"DELLEPIANE, LUIS, TTE. GRAL.","LACARRA AV. y DELLEPIANE, LUIS, TTE. GRAL.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,AUTO-AUTO,AUTO,AUTO


Verificamos valores nulos con la funcion verificar_tipo_datos



In [14]:
verificar_tipo_datos(df_hechos)

Total de valores nulos en el DataFrame: 747

Columnas con valores nulos y sus porcentajes:
             nombre_campo  nulos_%
11                 Altura    81.47
12                  Cruce    24.57
13  Dirección normalizada     1.15
10                  Calle     0.14


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Id,[<class 'str'>],100.00,0.00,0
1,N victimas,[<class 'int'>],100.00,0.00,0
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.00,0.00,0
3,Año,[<class 'int'>],100.00,0.00,0
4,Mes,[<class 'int'>],100.00,0.00,0
5,Día,[<class 'int'>],100.00,0.00,0
6,Hora,"[<class 'datetime.time'>, <class 'str'>, <clas...",100.00,0.00,0
7,Hora entera,"[<class 'int'>, <class 'str'>]",100.00,0.00,0
8,Lugar del hecho,[<class 'str'>],100.00,0.00,0
9,Tipo de calle,[<class 'str'>],100.00,0.00,0


Como podemos ver de todas las columnas tenemos 4 columnas con valores nulos las cuales son las siguientes:

Altura
Cruce
Dirección normalizada
Calle

Análisis columna Altura
Analizando la información dada, llegamos a la conclusion que:

Columna altura: no nos brinda una información que sea util, ya que la mayoria de los accidentes suceden en las esquinas ademas podemos encontrar el mismo dato en la columna Lugar del Hecho y Dirección Normalizada.

In [15]:
# Seleccionar e imprimir las primeras cinco filas de las columnas 'Altura', 'Lugar del hecho' y 'Dirección normalizada' en df_hechos.
df_hechos[['Altura', 'Lugar del hecho', 'Dirección normalizada']][:5]
     

,Altura,Lugar del hecho,Dirección normalizada
0,NaN,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G..."
1,NaN,AV GRAL PAZ Y AV DE LOS CORRALES,"PAZ, GRAL. AV. y DE LOS CORRALES AV."
2,2034.0,AV ENTRE RIOS 2034,ENTRE RIOS AV. 2034
3,NaN,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL."
4,NaN,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES."


Borramos la columna Altura

In [16]:
# Eliminar la columna 'Altura' del DataFrame df_hechos.
df_hechos = df_hechos.drop('Altura', axis=1)

Análisis columna Cruce


Nos enfocamos ahora en la columna Crucey podemos ver en el ejemplo de abajo que los valores que contienen "Nan" es por que no es un cruce entre dos arterias. Cambiamos los datos de la columna por "si" y por "no" refiriendose a que si es cruce o no.

In [17]:

# Seleccionamos las columnas 'Cruce', 'Lugar del hecho', y 'Tipo de calle' del DataFrame df_hechos
# y limitamos la visualización a las primeras 5 filas para una revisión preliminar concentrada
df_hechos[['Cruce', 'Lugar del hecho', 'Tipo de calle']][:5]

,Cruce,Lugar del hecho,Tipo de calle
0,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.",AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA
1,DE LOS CORRALES AV.,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ
2,NaN,AV ENTRE RIOS 2034,AVENIDA
3,"VILLEGAS, CONRADO, GRAL.",AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA
4,"SAENZ PE?A, LUIS, PRES.",AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA


In [18]:

# Reemplazar valores no nulos por "SI" y nulos por "NO" en la columna 'Cruce'
df_hechos['Cruce'] = np.where(df_hechos['Cruce'].notnull(), 'SI', 'NO')

# Revisar y seleccionar las columnas 'Cruce', 'Lugar del hecho', 'Tipo de calle' en el DataFrame
columnas_seleccionadas = ['Cruce', 'Lugar del hecho', 'Tipo de calle']
df_hechos_seleccion = df_hechos[columnas_seleccionadas]

In [19]:

# df_hechos_seleccion: Variable que se espera contenga un subconjunto filtrado o específicamente seleccionado del DataFrame df_hechos.
df_hechos_seleccion
     

,Cruce,Lugar del hecho,Tipo de calle
0,SI,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA
1,SI,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ
2,NO,AV ENTRE RIOS 2034,AVENIDA
3,SI,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA
4,SI,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA
...,...,...,...
691,SI,AV. RIESTRA Y MOM,AVENIDA
692,SI,AU DELLEPIANE Y LACARRA,AUTOPISTA
693,SI,AV. GAONA Y TERRADA,AVENIDA
694,NO,AV. EVA PERON 4071,AVENIDA


Análisis columna Dirección normalizada


Nos enfocamos ahora en la columna Dirección normalizada y vemos que solo tiene 8 datos nulos (un 1,15% de valores falantes). Lo examinamos:

In [20]:
df_hechos[df_hechos['Dirección normalizada'].isnull()][['Dirección normalizada', 'Lugar del hecho', 'Tipo de calle']][:10]

,Dirección normalizada,Lugar del hecho,Tipo de calle
38,NaN,AUTOPISTA LUGONES PK 10000,AUTOPISTA
106,NaN,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA
119,NaN,SD,CALLE
180,NaN,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA
181,NaN,AU DELLEPIANE 2400,AUTOPISTA
313,NaN,AUTOPISTA LUGONES KM 4.7,AUTOPISTA
546,NaN,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA
621,NaN,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA


Como no tenemos información para completar los registros pero siguen siendo registros, solo cargamos la columna con "SD"

In [21]:
# Se rellenan nulos con Falta info
df_hechos['Dirección normalizada'].fillna("SD", inplace=True)

# Se verifican los datos imputados
df_hechos[df_hechos['Dirección normalizada']== 'SD'][['Dirección normalizada', 'Lugar del hecho', 'Tipo de calle']]
     

,Dirección normalizada,Lugar del hecho,Tipo de calle
38,SD,AUTOPISTA LUGONES PK 10000,AUTOPISTA
106,SD,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA
119,SD,SD,CALLE
180,SD,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA
181,SD,AU DELLEPIANE 2400,AUTOPISTA
313,SD,AUTOPISTA LUGONES KM 4.7,AUTOPISTA
546,SD,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA
621,SD,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA


Análisis columna Id

Datos duplicados en la columna "Id"

In [22]:
# Imprimimos el número de valores únicos en la columna 'Id' del DataFrame df_hechos, lo cual nos da una idea de la unicidad de los registros
print(f"La columna 'Id' cuenta con {len(df_hechos['Id'].unique())} valores únicos.")
# Llamamos a la función verifica_duplicados_por_columna, pasando el DataFrame df_hechos y la columna 'Id' como argumentos
verifica_duplicados_por_columna(df_hechos, 'Id')

La columna 'Id' cuenta con 696 valores únicos.


'No hay duplicados'

La columna 'Id' cuenta con 696 valores únicos.

'No hay duplicados'

No centramos ahora en identificar el tipo de datos que contienen nuestras columnas, para ello llamamos a la función obtener_tipos_de_datos

In [23]:
obtener_tipos_de_datos(df_hechos)


,Columna,Tipo de dato
0,Id,[<class 'str'>]
1,N victimas,[<class 'int'>]
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...
3,Año,[<class 'int'>]
4,Mes,[<class 'int'>]
5,Día,[<class 'int'>]
6,Hora,"[<class 'datetime.datetime'>, <class 'str'>, <..."
7,Hora entera,"[<class 'str'>, <class 'int'>]"
8,Lugar del hecho,[<class 'str'>]
9,Tipo de calle,[<class 'str'>]


Análisis columna Hora


Vamos revisar las columnas: Hora, Hora entera y Calle por presentar distintos tipos de datos en la columna.

In [24]:

# Cantidad de valores por tipo de dato en la columna 'hora'
df_hechos['Hora'].apply(type).value_counts()

Hora
<class 'datetime.time'>        608
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

Por ser de mayor cantidad convertimos todo al tipo de datos "datetime.time".

In [25]:
# Cambiamos el tipo de dato de la columna 'Hora' aplicando la función convertir_tiempo a cada elemento.
# Esta función ajusta los valores a un formato de tiempo estandarizado o retorna None si la conversión no es posible.
df_hechos['Hora'] = df_hechos['Hora'].apply(lambda x: convertir_tiempo(x))

# Imprimimos la cantidad de valores por tipo de dato en la columna 'Hora' para verificar la conversión
# y entender la distribución de tipos de datos resultantes.
print('Tipos de datos:')
print(df_hechos['Hora'].apply(type).value_counts())

# Imprimimos y buscamos registros donde la columna 'Hora' es de tipo NoneType,
# lo que indica que la conversión no fue posible para esos registros.
print('Registro con NoneType:')
df_hechos[df_hechos['Hora'].isna()]


Tipos de datos:
Hora
<class 'datetime.time'>    695
<class 'NoneType'>           1
Name: count, dtype: int64
Registro con NoneType:


,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,None,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Para rellenar con un valor la columna Hora y Hora entera decidimos que se va a sacar la moda de la columna

In [26]:

# calcula la hora mas común
hora_moda = df_hechos['Hora'].mode().iloc[0]
print(f'La hora mas común es: {hora_moda}')

# Se reemplaza el valor None por la hora más común
df_hechos['Hora'].fillna(hora_moda, inplace=True)

# se verifica el tipo de dato para la columna
df_hechos['Hora'].apply(type).value_counts()

La hora mas común es: 09:00:00


Hora
<class 'datetime.time'>    696
Name: count, dtype: int64

Análisis columna Hora entera


Ahora analizamos la columna Hora entera que presentab valores SD (sin dato) por lo tanto hacemos lo mismo, le imputamos la hora mas común.

In [27]:

# Se verifica el tipo de dato
df_hechos['Hora entera'].apply(type).value_counts()

Hora entera
<class 'int'>    695
<class 'str'>      1
Name: count, dtype: int64

In [28]:
# Se imputa la hora moda al dato faltante
df_hechos['Hora entera'] = df_hechos['Hora entera'].apply(lambda x: int(hora_moda.hour) if x == "SD" else x)
# Se verifica el tipo de dato
df_hechos['Hora entera'].apply(type).value_counts()

Hora entera
<class 'int'>    696
Name: count, dtype: int64

In [29]:
# Se verifica el registro completado
df_hechos[df_hechos['Id']=='2019-0103']

,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
518,2019-0103,1,2019-12-18,2019,12,18,09:00:00,9,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Análisis columna Calle


Finalmente, la columna 'Calle' presenta dos tipos de datos. Por la información de la variable, es una columna de strings pero se presentan algunos float. Se revisan los registros con este último tipo de dato.

In [30]:

# Se verifica el tipo de dato
print('Tipos de datos:')
print(df_hechos['Hora entera'].apply(type).value_counts())
# Se observa el registro con tipo de dato float
print('Registro con tipo float:')
df_hechos[df_hechos['Calle'].apply(lambda x: isinstance(x, float))]
     

Tipos de datos:
Hora entera
<class 'int'>    696
Name: count, dtype: int64
Registro con tipo float:


,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,NaN,NO,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD


Se observa que no se cuenta con el dato de la calle donde ocurrió el hecho. Por lo tanto, se decide imputar 'SD' a este valor faltante.



In [31]:
# Se imputa SD al dato faltante
df_hechos['Calle'].fillna('SD', inplace=True)
# Se verifica el registro completado
df_hechos[df_hechos['Id']=='2016-0151']

,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,NO,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD


Verificamos como nos quedaron los tipos de datos



In [32]:
verificar_tipo_datos(df_hechos)


Total de valores nulos en el DataFrame: 0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Id,[<class 'str'>],100.0,0.0,0
1,N victimas,[<class 'int'>],100.0,0.0,0
2,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
3,Año,[<class 'int'>],100.0,0.0,0
4,Mes,[<class 'int'>],100.0,0.0,0
5,Día,[<class 'int'>],100.0,0.0,0
6,Hora,[<class 'datetime.time'>],100.0,0.0,0
7,Hora entera,[<class 'int'>],100.0,0.0,0
8,Lugar del hecho,[<class 'str'>],100.0,0.0,0
9,Tipo de calle,[<class 'str'>],100.0,0.0,0


Análisis columna Víctima y Acusado


Ahora revisamos la categoria Victima y Acusado

In [33]:
df_hechos['Víctima'].unique()


array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

In [34]:
df_hechos['Acusado'].unique()


array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

Para 'Acusado' no se observan características distintas a las esperadas. En cambio, para las categorías de 'Víctima' se observan dos categorías Objeto fijo y Peatón moto que no están en el diccionario de datos. Se observan estos registros para corregirlos.

In [35]:
df_hechos[df_hechos['Víctima'].isin(['OBJETO FIJO', 'PEATON_MOTO'])]


,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,SI,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


Se observan dos registros, uno por cada término distinto al del diccionario de datos. Se decide cambiarlo por OTRO dado que no hay otro dato que pueda indicar el vehículo de la victima.

In [36]:

# Se cambia por OTRO
df_hechos['Víctima'] = df_hechos['Víctima'].replace({'OBJETO FIJO':'OTRO', 'PEATON_MOTO':'OTRO'})

# Se verifican esos cambios
df_hechos[df_hechos['Id'].isin(['2017-0108', '2020-0063'])]

,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OTRO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,SI,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,OTRO,MOTO


Analizamos ahora las variables de ubicación


Anteriormente pudimos ver faltante de información en las columnas geográficas XY_(CABA), Pos_x y Pos_y.

In [37]:

# Filtramos el DataFrame df_hechos para encontrar registros donde la columna 'XY (CABA)' tenga el valor específico 'Point (. .)'
# Esto nos permite identificar registros con posibles problemas en los datos de ubicación.
df_hechos[df_hechos['XY (CABA)']=='Point (. .)']


,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NO,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NO,SD,13,Point (. .),.,.,MOTO-SD,MOTO,SD
71,2016-0096,1,2016-07-25,2016,7,25,07:00:00,7,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NO,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,8,Point (. .),-58.47433193007387,-34.66684950051973,MOTO-CARGAS,MOTO,CARGAS
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NO,SD,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,NO,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,NO,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",NO,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,NO,SD,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NO,SD,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NO,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS


Son 14 registros que no presentan las coordenadas planas del hecho y también se ven 10 registros que tampoco presentan los datos 'Pos x' y 'Pos y'. Se buscó información que permitiera completar estos datos, pero al momento de cierre del presente informe aún no se encontró. Para continuar con el análsis se imputarán 0 de modo tal que sirva para excluir el valor en el dashboard a realizar.

In [38]:

# Se reemplazan los valores faltantes por 0
df_hechos['Pos x'] = df_hechos['Pos x'].replace('.', 0)
df_hechos['Pos y'] = df_hechos['Pos y'].replace('.', 0)

In [39]:

# Reemplazamos en la columna 'XY (CABA)' del DataFrame df_hechos los valores mal formateados 'Point (. .)' por 0.
# Esto estandariza los datos de ubicación y marca los registros con información de ubicación faltante o errónea.
df_hechos['XY (CABA)'] = df_hechos['XY (CABA)'].replace('Point (. .)', 0)
     

In [40]:
# Buscamos en el DataFrame df_hechos para confirmar que ya no existen registros con la columna 'XY (CABA)' igual a 'Point (. .)'
# Este paso verifica la efectividad de la corrección aplicada previamente, asegurando que los valores mal formateados han sido adecuadamente reemplazados.
df_hechos[df_hechos['XY (CABA)']=='Point (. .)']

,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado


Verificamos como quedaron los datos en el Id 2016-0052



In [41]:

# Buscamos en el DataFrame df_hechos el registro específico cuyo 'Id' es igual a '2016-0052'.
# Esto nos permite examinar detalladamente las características y los datos asociados a este caso en particular.
df_hechos[df_hechos['Id']=='2016-0052']

,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NO,SD,13,0,0,0,MOTO-SD,MOTO,SD


Analizamos la pestanaña Víctimas con el dataframe df_Víctimas
Info del dataframe
Se extraen los datos de la pestaña VICTIMAS del excel homicidios y se observan algunas filas.

In [42]:
# Selecciona 5 filas aleatorias de 'df_victimas' con reproducibilidad
df_victimas.sample(5, random_state=5)
     

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
521,2019-0086,2019-10-15,2019,10,15,CONDUCTOR,MOTO,MASCULINO,29,2019-10-15 00:00:00
698,2021-0079,2021-10-25,2021,10,25,PEATON,PEATON,MASCULINO,SD,2021-10-25 00:00:00
300,2018-0015,2018-02-06,2018,2,6,CONDUCTOR,MOTO,MASCULINO,28,2018-02-06 00:00:00
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
23,2016-0034,2016-03-12,2016,3,12,PEATON,PEATON,MASCULINO,52,2016-03-12 00:00:00


El conjunto de datos presenta 10 variables (columnas) y 717 registros lo que se pueden ver a continuación.



In [43]:
# Muestra un resumen conciso del DataFrame incluyendo cantidad de entradas no nulas y tipo de dato por columna
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Dado que algunas columnas estan nombradas en minúsculas ymayúsculas renombramos las columnas y las dejamos a todas estandarizadas

In [44]:

# Convierte la primera letra de cada nombre de columna en mayúscula
df_victimas.columns = [x.capitalize() for x in df_victimas.columns]
# Reemplaza guiones bajos por espacios en los nombres de las columnas
df_victimas.columns = df_victimas.columns.str.replace('_', ' ')
# Renombra específicamente columnas para claridad y consistencia
df_victimas = df_victimas.rename(columns={'Id hecho': 'Id',
                                          'Aaaa':'Año',
                                          'Mm':'Mes',
                                          'Dd':'Día',
                                          'Victima':'Víctima'})
# Muestra los nombres actuales de las columnas
df_victimas.columns

Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Víctima', 'Sexo', 'Edad',
       'Fecha fallecimiento'],
      dtype='object')

Verificamos valores nulos

In [45]:
# Verifica si hay valores nulos en homicidios_victimas
df_victimas.isna().any().any()

False

Verificamos datos duplicados


In [46]:
# Se buscan las filas repetidas
df_victimas[df_victimas.duplicated()]
     

,Id,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento


Verificamos tipos devariables


In [47]:
# Se verican las columnas por tipo de dato
verificar_tipo_datos(df_victimas)

Total de valores nulos en el DataFrame: 0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Id,[<class 'str'>],100.0,0.0,0
1,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
2,Año,[<class 'int'>],100.0,0.0,0
3,Mes,[<class 'int'>],100.0,0.0,0
4,Día,[<class 'int'>],100.0,0.0,0
5,Rol,[<class 'str'>],100.0,0.0,0
6,Víctima,[<class 'str'>],100.0,0.0,0
7,Sexo,[<class 'str'>],100.0,0.0,0
8,Edad,"[<class 'int'>, <class 'str'>]",100.0,0.0,0
9,Fecha fallecimiento,"[<class 'datetime.datetime'>, <class 'str'>]",100.0,0.0,0


Total de valores nulos en el DataFrame: 0


Análisis de columna Edad


Verificamos los tipos y cantidades de datos de la columna

In [48]:
# Cantidad de valores por tipo de dato en la columna 'edad'
tipos_datos = df_victimas['Edad'].apply(type).value_counts()
print('Los tipos de datos son:')
print(tipos_datos)
print(f'Los datos {tipos_datos.index[1]} representan el {round((tipos_datos[1]/tipos_datos.sum()),2)}')

Los tipos de datos son:
Edad
<class 'int'>    664
<class 'str'>     53
Name: count, dtype: int64
Los datos <class 'str'> representan el 0.07


Podemos ver 664 registros de tipo entero y 53 registros de tipo string que representa el 7% de los registros. Vamos a revisar ese 7%.



In [49]:
df_victimas[df_victimas['Edad'].apply(lambda x: isinstance(x, str))]


,Id,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,SD,SD
35,2016-0048,2016-04-15,2016,4,15,PEATON,PEATON,FEMENINO,SD,SD
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,SD
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,SD
55,2016-0077,2016-06-13,2016,6,13,PEATON,PEATON,FEMENINO,SD,SD
63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,SD,SD
72,2016-0096,2016-07-25,2016,7,25,CONDUCTOR,MOTO,MASCULINO,SD,SD
89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,SD
93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,SD,FEMENINO,SD,SD


Para poder rellenar esos datos faltantes, se quiere imputar los mismos con el promedio de las edades de las victimas, pero teniendo en cuenta el 'Sexo'. Como se observa que esa variable también presenta faltantes de datos, primero se completan esos valores, teniendo en cuenta el valor mas frecuente entre las víctimas. Luegoimpitamos la edad segun el sexo

In [50]:
# Imputa valor mas frecuente en la columna sexo
imputa_valor_frecuente(df_victimas, 'Sexo')
     

El valor mas frecuente es: MASCULINO


El valor mas frecuente es: MASCULINO


In [51]:

# Imputa la edad segun el sexo
imputa_edad_media_segun_sexo(df_victimas)
     

La edad promedio de Femenino es 51 y de Masculino es 40


La edad promedio de Femenino es 51 y de Masculino es 40


Chequeamos algunos ejemplos



In [52]:

id_hecho_ejemplos = ['2016-0045', '2016-0048', '2016-0049']

df_victimas[df_victimas['Id'].isin(id_hecho_ejemplos)]

,Id,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,39,SD
35,2016-0048,2016-04-15,2016,4,15,PEATON,PEATON,FEMENINO,50,SD
36,2016-0049,2016-04-17,2016,4,17,SD,SD,MASCULINO,39,SD


Análisis de columna Rol y Víctima


En ese ejemplo se observa que 'Rol' y 'Victima' también tiene faltantes de datos. Se revisan la cantidad de SD que hay en cada una.

In [53]:

print(f"La cantidad de SD en 'rol' es de {len(df_victimas[df_victimas['Rol']=='SD'])}")
print(f"La cantidad de SD en 'victima' es de {len(df_victimas[df_victimas['Víctima']=='SD'])}")

La cantidad de SD en 'rol' es de 11
La cantidad de SD en 'victima' es de 9


Al comprobarse que son pocos los datos donde falta la información, se decide imputar el valor mas frecuente para cada columna.

In [54]:

# Imputa valor mas frecuente en la columna rol
imputa_valor_frecuente(df_victimas, 'Rol')

El valor mas frecuente es: CONDUCTOR


El valor mas frecuente es: CONDUCTOR


In [55]:

# Imputa valor mas frecuente en la columna victima
imputa_valor_frecuente(df_victimas, 'Víctima')

El valor mas frecuente es: MOTO


Se revisan los mismos ejemplos anteriores, para ver estas últimas imputaciones.



In [56]:

id_hecho_ejemplos = ['2016-0045', '2016-0048', '2016-0049']
df_victimas[df_victimas['Id'].isin(id_hecho_ejemplos)]
     

,Id,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad,Fecha fallecimiento
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,39,SD
35,2016-0048,2016-04-15,2016,4,15,PEATON,PEATON,FEMENINO,50,SD
36,2016-0049,2016-04-17,2016,4,17,CONDUCTOR,MOTO,MASCULINO,39,SD


Análisis de columna Fecha fallecimiento


Finalmente, queda la columna 'Fecha fallecimiento' que presenta dos tipo de datos y por lo que se observa de los ejemplos anteriores, se refiere a faltantes de datos. Como el objetivo es analizar los datos para encontrar patrones que permitan tomar decisiones para disminuir los accidentes fatales, conocer la fecha de fallecimiento no aporta información al análsis, por lo que se decide borrar la column

In [57]:

df_victimas = df_victimas.drop('Fecha fallecimiento', axis=1)
df_victimas.head(3)

,Id,Fecha,Año,Mes,Día,Rol,Víctima,Sexo,Edad
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30


Se verifica una vez mas el tipo de datos completo del conjunto de datos.



In [58]:

# Se verican las columnas por tipo de dato
verificar_tipo_datos(df_victimas)


Total de valores nulos en el DataFrame: 0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Id,[<class 'str'>],100.0,0.0,0
1,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
2,Año,[<class 'int'>],100.0,0.0,0
3,Mes,[<class 'int'>],100.0,0.0,0
4,Día,[<class 'int'>],100.0,0.0,0
5,Rol,[<class 'str'>],100.0,0.0,0
6,Víctima,[<class 'str'>],100.0,0.0,0
7,Sexo,[<class 'str'>],100.0,0.0,0
8,Edad,[<class 'int'>],100.0,0.0,0


Total de valores nulos en el DataFrame: 0


Eliminación de columnas


Dado que este conjunto de datos se unirá con el homicidios_hechos, se pueden eliminar las columnas 'Fecha', 'Año', 'Mes', 'Día' y 'Víctima', porque es la misma información que contiene homicidios_hechos. Se verifica un ejemplo.

In [59]:

df_hechos[df_hechos['Id']=='2016-0001']

,Id,N victimas,Fecha,Año,Mes,Día,Hora,Hora entera,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,SI,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


In [60]:
# Se eliminan las columnas repetidas
df_victimas = df_victimas.drop(['Fecha', 'Año', 'Mes', 'Día', 'Víctima'], axis=1)
df_victimas.columns

Index(['Id', 'Rol', 'Sexo', 'Edad'], dtype='object')

In [61]:
# Vemos un ejemplo como queda
df_victimas.head()

,Id,Rol,Sexo,Edad
0,2016-0001,CONDUCTOR,MASCULINO,19
1,2016-0002,CONDUCTOR,MASCULINO,70
2,2016-0003,CONDUCTOR,MASCULINO,30
3,2016-0004,CONDUCTOR,MASCULINO,18
4,2016-0005,CONDUCTOR,MASCULINO,29


Unificación del dataframe df_hechos y df_victimas

Se unifican en un único dataframe los dos conjuntos de datos ingestados anteriormente.

In [62]:
df_homicidios = df_victimas.merge(df_hechos, on='Id', how='left')
df_homicidios

,Id,Rol,Sexo,Edad,N victimas,Fecha,Año,Mes,Día,Hora,...,Calle,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado
0,2016-0001,CONDUCTOR,MASCULINO,19,1,2016-01-01,2016,1,1,04:00:00,...,PIEDRA BUENA AV.,SI,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,CONDUCTOR,MASCULINO,70,1,2016-01-02,2016,1,2,01:15:00,...,"PAZ, GRAL. AV.",SI,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,CONDUCTOR,MASCULINO,30,1,2016-01-03,2016,1,3,07:00:00,...,ENTRE RIOS AV.,NO,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,CONDUCTOR,MASCULINO,18,1,2016-01-10,2016,1,10,00:00:00,...,LARRAZABAL AV.,SI,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,CONDUCTOR,MASCULINO,29,1,2016-01-21,2016,1,21,05:20:00,...,SAN JUAN AV.,SI,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,PEATON,FEMENINO,50,1,2021-12-12,2021,12,12,06:20:00,...,RIVADAVIA AV.,SI,RIVADAVIA AV. y PUEYRREDON AV.,3,Point (105258.35368554 102122.93231400),-58.40596860,-34.61011987,PEATON-AUTO,PEATON,AUTO
713,2021-0093,PASAJERO_ACOMPAÑANTE,FEMENINO,18,1,2021-12-13,2021,12,13,17:10:00,...,RIESTRA AV.,SI,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
714,2021-0094,PASAJERO_ACOMPAÑANTE,FEMENINO,43,1,2021-12-20,2021,12,20,01:10:00,...,"DELLEPIANE, LUIS, TTE. GRAL.",SI,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
715,2021-0095,CONDUCTOR,MASCULINO,27,1,2021-12-30,2021,12,30,00:43:00,...,GAONA AV.,SI,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS


In [63]:
# Convertir las columnas 'Pos x' y 'Pos y' a tipo float
df_homicidios['Pos x'] = df_homicidios['Pos x'].astype(float)
df_homicidios['Pos y'] = df_homicidios['Pos y'].astype(float)

In [64]:
verificar_tipo_datos(df_homicidios)


Total de valores nulos en el DataFrame: 0


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Id,[<class 'str'>],100.0,0.0,0
1,Rol,[<class 'str'>],100.0,0.0,0
2,Sexo,[<class 'str'>],100.0,0.0,0
3,Edad,[<class 'int'>],100.0,0.0,0
4,N victimas,[<class 'int'>],100.0,0.0,0
5,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
6,Año,[<class 'int'>],100.0,0.0,0
7,Mes,[<class 'int'>],100.0,0.0,0
8,Día,[<class 'int'>],100.0,0.0,0
9,Hora,[<class 'datetime.time'>],100.0,0.0,0


Total de valores nulos en el DataFrame: 0

Se verifican los tipos de datos y cantidad de nulos en este nuevo dataframe.

No hay varios tipos de datos en las columnas y no se presentan valores nulos. A continuación, se trabaja con este conjunto de datos para analizarlo en pos de encontrar patrones que perminan hacer recomendaciones para disminuir la cantidad de accidentes fatales.

Cargamos Población


In [65]:
df_comunas = pd.read_excel(r'C:\Users\Moises\Desktop\Proyecto individual 2\archivos\comunas.xlsx',sheet_name='Comunas')
df_comunas.sample(14)

,OBJETO,Comuna,BARRIOS,PERIMETRO,AREA
3,LIMITE COMUNAL,4,BARRACAS - BOCA - NUEVA POMPEYA - PARQUE PATRI...,35423.282799,2.168464e+07
1,LIMITE COMUNAL,2,RECOLETA,21452.838648,6.317265e+06
2,LIMITE COMUNAL,3,BALVANERA - SAN CRISTOBAL,10486.260809,6.385991e+06
7,LIMITE COMUNAL,8,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,20935.012513,2.221690e+07
5,LIMITE COMUNAL,6,CABALLITO,10990.964471,6.851029e+06
14,LIMITE COMUNAL,15,AGRONOMIA - CHACARITA - PARQUE CHAS - PATERN...,17832.585580,1.432290e+07
9,LIMITE COMUNAL,10,FLORESTA - MONTE CASTRO - VELEZ SARSFIELD - VE...,18332.037457,1.265656e+07
4,LIMITE COMUNAL,5,ALMAGRO - BOEDO,12323.432479,6.660603e+06
12,LIMITE COMUNAL,13,BELGRANO - COLEGIALES - NUÃ‘EZ,26198.826953,1.471321e+07
8,LIMITE COMUNAL,9,LINIERS - MATADEROS - PARQUE AVELLANEDA,21411.738344,1.650531e+07


Juntamos el dataframe df_homicidios junto con el dataframe df_comunas para ver ademas del número de comunas tambien los barrios que la componen.

In [66]:
df_homicidios2 = pd.merge(df_homicidios, df_comunas, on="Comuna", how="left")


In [67]:
verificar_tipo_datos(df_homicidios2)


Total de valores nulos en el DataFrame: 8

Columnas con valores nulos y sus porcentajes:
   nombre_campo  nulos_%
23       OBJETO     0.28
24      BARRIOS     0.28
25    PERIMETRO     0.28
26         AREA     0.28


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,Id,[<class 'str'>],100.00,0.00,0
1,Rol,[<class 'str'>],100.00,0.00,0
2,Sexo,[<class 'str'>],100.00,0.00,0
3,Edad,[<class 'int'>],100.00,0.00,0
4,N victimas,[<class 'int'>],100.00,0.00,0
5,Fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.00,0.00,0
6,Año,[<class 'int'>],100.00,0.00,0
7,Mes,[<class 'int'>],100.00,0.00,0
8,Día,[<class 'int'>],100.00,0.00,0
9,Hora,[<class 'datetime.time'>],100.00,0.00,0


Borramos las columnas que no nos interesan para el análisis que son las columnas "OBJETO", "PERIMETRO", "AREA"

In [68]:

columnas_a_borrar = ["OBJETO", "PERIMETRO", "AREA"]
df_homicidios2 = df_homicidios2.drop(columnas_a_borrar, axis=1)
df_homicidios2 = df_homicidios2.dropna()

Guardamos como 1_homicidios_limpio

In [69]:

#Nombramos nuevamente df_homcidios con el dataframe principal
df_homicidios = df_homicidios2

In [70]:
#Visualizamos un ejemplo
df_homicidios.sample(5)

,Id,Rol,Sexo,Edad,N victimas,Fecha,Año,Mes,Día,Hora,...,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado,BARRIOS
203,2017-0068,CONDUCTOR,MASCULINO,32,1,2017-05-27,2017,5,27,19:50:00,...,NO,"RABANAL, FRANCISCO, INTENDENTE AV. 2950",8,Point (102520.03961549 96105.29592085),-58.435807,-34.664375,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI
193,2017-0058,PEATON,MASCULINO,80,1,2017-05-09,2017,5,9,23:10:00,...,SI,CORDOBA AV. y LIBERTAD,1,Point (107271.27725903 103337.46347012),-58.384032,-34.599159,PEATON-MOTO,PEATON,MOTO,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...
79,2016-0104,CONDUCTOR,MASCULINO,21,1,2016-08-14,2016,8,14,21:15:00,...,SI,ZINNY y SANTIAGO DE COMPOSTELA AV.,9,Point (99720.41444035 97007.69579491),-58.466350,-34.656244,MOTO-PASAJEROS,MOTO,PASAJEROS,LINIERS - MATADEROS - PARQUE AVELLANEDA
440,2019-0006,PEATON,FEMENINO,12,1,2019-01-23,2019,1,23,05:45:00,...,NO,"OBLIGADO RAFAEL, AV.COSTANERA 4650",14,Point (104292.73762877 108227.37900690),-58.416527,-34.555096,PEATON-AUTO,PEATON,AUTO,PALERMO
30,2016-0041,PASAJERO_ACOMPAÑANTE,MASCULINO,39,2,2016-03-29,2016,3,29,11:00:00,...,SI,DIRECTORIO AV. y RIVERA INDARTE AV.,7,Point (100232.38564985 99530.25843190),-58.460766,-34.633504,MOTO-CARGAS,MOTO,CARGAS,FLORES - PARQUE CHACABUCO


In [71]:
# Dirección donde deseas guardar el archivo Excel
direccion_excel = r'C:\Users\Moises\Desktop\Proyecto individual 2\archivos\siniestros_limpios.csv'

# Guardar el DataFrame como un archivo Excel en la dirección especificada
df_homicidios.to_csv(direccion_excel, index=True)


KPI 1

In [72]:
siniestros = pd.read_csv(r'C:\Users\Moises\Desktop\Proyecto individual 2\archivos\siniestros_limpios.csv')
siniestros

,Unnamed: 0,Id,Rol,Sexo,Edad,N victimas,Fecha,Año,Mes,Día,...,Cruce,Dirección normalizada,Comuna,XY (CABA),Pos x,Pos y,Participantes,Víctima,Acusado,BARRIOS
0,0,2016-0001,CONDUCTOR,MASCULINO,19,1,2016-01-01,2016,1,1,...,SI,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI
1,1,2016-0002,CONDUCTOR,MASCULINO,70,1,2016-01-02,2016,1,2,...,SI,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,LINIERS - MATADEROS - PARQUE AVELLANEDA
2,2,2016-0003,CONDUCTOR,MASCULINO,30,1,2016-01-03,2016,1,3,...,NO,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...
3,3,2016-0004,CONDUCTOR,MASCULINO,18,1,2016-01-10,2016,1,10,...,SI,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,SD,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI
4,4,2016-0005,CONDUCTOR,MASCULINO,29,1,2016-01-21,2016,1,21,...,SI,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,712,2021-0092,PEATON,FEMENINO,50,1,2021-12-12,2021,12,12,...,SI,RIVADAVIA AV. y PUEYRREDON AV.,3,Point (105258.35368554 102122.93231400),-58.405969,-34.610120,PEATON-AUTO,PEATON,AUTO,BALVANERA - SAN CRISTOBAL
711,713,2021-0093,PASAJERO_ACOMPAÑANTE,FEMENINO,18,1,2021-12-13,2021,12,13,...,SI,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO,FLORES - PARQUE CHACABUCO
712,714,2021-0094,PASAJERO_ACOMPAÑANTE,FEMENINO,43,1,2021-12-20,2021,12,20,...,SI,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO,LINIERS - MATADEROS - PARQUE AVELLANEDA
713,715,2021-0095,CONDUCTOR,MASCULINO,27,1,2021-12-30,2021,12,30,...,SI,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS,VILLA DEL PARQUE - VILLA DEVOTO - VILLA GRAL....


In [73]:
#Hago una copia para despues usar el mismo con el kpi_2
kpi_1 = siniestros.copy()

In [74]:
#Hago el semestre
kpi_1["SEMESTRE"] = (kpi_1["Mes"] - 1) // 6 + 1

In [75]:
#Agrupo Año y Semestre con el numero de victimas
#kpi_1 = kpi_1.groupby(["Año", "SEMESTRE"]).size().reset_index(name="Num víctimas")
kpi_1 = kpi_1.groupby(["Año", "SEMESTRE"])["N victimas"].sum().reset_index(name="Total víctimas por semestre")

In [76]:

#El semestre anterior lo hago con shift(), mueve el numero de victimas uno mas y si es el primero con fill_value=0 le agrega 0
kpi_1["SEMESTRE_ANTERIOR"] = kpi_1["Total víctimas por semestre"].shift(periods=1, fill_value=0)

In [77]:
#Hago las Tasas, con la formula que esta en el repo.
#La cantidad de personas lo saque de un senso.
kpi_1["TASA_HOMICIDIOS"] = (kpi_1["Total víctimas por semestre"]/3120612) * 100000
kpi_1["TASA_SEMESTRE_ANTERIOR"] = (kpi_1["SEMESTRE_ANTERIOR"]/3120612) * 100000

In [78]:
#Hago una columna llamada variacion con las Tasas, es el porcentaje de que tanto cambiaron de un semestre a otro.

#kpi_1["VARIACION"] = (kpi_1["Total víctimas por semestre"] - kpi_1["SEMESTRE_ANTERIOR"]) / kpi_1["SEMESTRE_ANTERIOR"] * 100
kpi_1["VARIACION"] = ((kpi_1["TASA_HOMICIDIOS"] - kpi_1["TASA_SEMESTRE_ANTERIOR"]) / kpi_1["TASA_SEMESTRE_ANTERIOR"]) * 100

In [79]:
kpi_1

,Año,SEMESTRE,Total víctimas por semestre,SEMESTRE_ANTERIOR,TASA_HOMICIDIOS,TASA_SEMESTRE_ANTERIOR,VARIACION
0,2016,1,67,0,2.147015,0.000000,inf
1,2016,2,81,67,2.595645,2.147015,20.895522
2,2017,1,81,81,2.595645,2.595645,0.000000
3,2017,2,79,81,2.531555,2.595645,-2.469136
4,2018,1,76,79,2.435420,2.531555,-3.797468
5,2018,2,85,76,2.723825,2.435420,11.842105
6,2019,1,59,85,1.890655,2.723825,-30.588235
7,2019,2,47,59,1.506115,1.890655,-20.338983
8,2020,1,31,47,0.993395,1.506115,-34.042553
9,2020,2,56,31,1.794520,0.993395,80.645161


In [80]:
kpi_1.to_csv(r"C:\Users\Moises\Desktop\Proyecto individual 2\archivos\kpi_1.csv", index=False)

KPI 2

In [81]:
#Repito pero con el repositorio 2
kpi_2 = siniestros.copy()

In [82]:
kpi_2.columns

Index(['Unnamed: 0', 'Id', 'Rol', 'Sexo', 'Edad', 'N victimas', 'Fecha', 'Año',
       'Mes', 'Día', 'Hora', 'Hora entera', 'Lugar del hecho', 'Tipo de calle',
       'Calle', 'Cruce', 'Dirección normalizada', 'Comuna', 'XY (CABA)',
       'Pos x', 'Pos y', 'Participantes', 'Víctima', 'Acusado', 'BARRIOS'],
      dtype='object')

In [83]:
kpi_2 = kpi_2[kpi_2["Víctima"] == "MOTO"]


In [84]:
kpi_2 = kpi_2.groupby(["Año"])["N victimas"].sum().reset_index(name="Total año victimas")


In [85]:
kpi_2["AÑO_ANTERIOR"] = kpi_2["Total año victimas"].shift(periods=1, fill_value=0)


In [86]:
kpi_2["TASA_HOMICIDIOS"] = (kpi_2["Total año victimas"]/3120612) * 100000
kpi_2["TASA_AÑO_ANTERIOR"] = (kpi_2["AÑO_ANTERIOR"]/3120612) * 100000

In [87]:

kpi_2["VARIACION"] = (kpi_2["TASA_HOMICIDIOS"] - kpi_2["TASA_AÑO_ANTERIOR"]) / kpi_2["TASA_AÑO_ANTERIOR"] * 100

In [88]:
kpi_2

,Año,Total año victimas,AÑO_ANTERIOR,TASA_HOMICIDIOS,TASA_AÑO_ANTERIOR,VARIACION
0,2016,67,0,2.147015,0.000000,inf
1,2017,62,67,1.986790,2.147015,-7.462687
2,2018,61,62,1.954745,1.986790,-1.612903
3,2019,50,61,1.602250,1.954745,-18.032787
4,2020,30,50,0.961350,1.602250,-40.000000
5,2021,46,30,1.474070,0.961350,53.333333


In [89]:
kpi_2.to_csv(r"C:\Users\Moises\Desktop\Proyecto individual 2\archivos\kpi_2.csv", index=False)

KPI 3


KPI 3: Reducir en un 5% la cantidad de accidentes mortales de peatones en el último año respecto al año ante

In [90]:
kpi_3 = siniestros.copy()


In [91]:
kpi_3 = kpi_3[kpi_3["Víctima"] == "PEATON"]


In [92]:
kpi_3 = kpi_3.groupby(["Año"])["N victimas"].sum().reset_index(name="Total victima peaton")


In [93]:
kpi_3["AÑO_ANTERIOR"] = kpi_3["Total victima peaton"].shift(periods=1, fill_value=0)


In [94]:
kpi_3["TASA_HOMICIDIOS"] = (kpi_3["Total victima peaton"]/3120612) * 100000
kpi_3["TASA_AÑO_ANTERIOR"] = (kpi_3["AÑO_ANTERIOR"]/3120612) * 100000

In [95]:
kpi_3["VARIACION"] = (kpi_3["TASA_HOMICIDIOS"] - kpi_3["TASA_AÑO_ANTERIOR"]) / kpi_3["TASA_AÑO_ANTERIOR"] * 100

In [96]:
kpi_3

,Año,Total victima peaton,AÑO_ANTERIOR,TASA_HOMICIDIOS,TASA_AÑO_ANTERIOR,VARIACION
0,2016,48,0,1.538160,0.000000,inf
1,2017,49,48,1.570205,1.538160,2.083333
2,2018,68,49,2.179060,1.570205,38.775510
3,2019,37,68,1.185665,2.179060,-45.588235
4,2020,34,37,1.089530,1.185665,-8.108108
5,2021,33,34,1.057485,1.089530,-2.941176


In [97]:
kpi_3.to_csv(r"C:\Users\Moises\Desktop\Proyecto individual 2\archivos\kpi_3.csv", index=False)